In [ ]:
!pip install python-dotenv

In [ ]:
import os
import weaviate
from askem. import load_dotenv

load_dotenv()
weaviate_apikey = os.getenv("WEAVIATE_APIKEY")
url = os.getenv("WEAVIATE_URL")
client = weaviate.Client(
    url,
    auth_client_secret=weaviate.auth.AuthApiKey(weaviate_apikey),
)

# How many paragraphs we have in the database?
client.query.aggregate("passage").with_meta_count().do()


In [ ]:
# Ask a sample question
question = "What is Antibacterial Activity?"

results = (
    client.query.get(
        "Passage", ["paper_id", "text_content", "topic", "preprocessor_id"]
    )
    .with_additional(["distance"])
    .with_near_text({"concepts": [question], "distance": 0.7})
    .with_limit(3)
    .do()
)

In [ ]:
ys = results["data"]["Get"]["Passage"]

In [ ]:
ys

In [ ]:
# What are the related paragraphs?
for x in result["data"]["Get"]["Passage"]:
    print(x["text_content"], "\n", "-" * 180)

In [ ]:
raw_vec = result["data"]["Get"]["Passage"][0]["_additional"]["vector"]
raw_vec

## With pre-filtering

In [ ]:
where_filter = {
    "path": ["topic"],
    "operator": "Equal",
    "valueText": "covid-19",
}

# where_filter = {
#     "path": ["preprocessor_id"],
#     "operator": "Equal",
#     "valueText": "haystack-v0.0.1",
# }


result = (
    client.query.get("Passage", ["text_content", "topic", "preprocessor_id"])
    .with_where(where_filter)
    .with_near_text(question)
    .with_limit(10)
    .with_additional(["vector", "distance", "id"])
    .do()
)